# Ciência dos Dados 2020 - Projeto 2
___
- Giovanni Rozatti
- Larissa Oliveira da Silva
___

<font color='#DF2535'>  

## Disclaimer:  
  
A ideia de criar um notebook organizacional foi herdada do P3 feito por Giovanni Rozatti (em conjunto com Daniel G. Terra e Rafael B. Zanfolin) no semestre anterior, o repositório para esse notebook daquele projeto se encontra no link a seguir:`https://github.com/VonIgnia/C.Dados-P3/blob/master/Classificador2/_Fun%C3%A7%C3%B5es.ipynb`
- A ideia original pertence a Daniel Terra, e sua execução(no Projeto 3) foi feita em conjunto com Giovanni Rozatti
- A maior parte das funções nesse notebook é inédita
- Partes diretmente extraídas daquele serão antecedidas por outro disclaimer  
</font>

### Objetivos do Notebook:
- O principal propósito desse notebook é a hospedagem e organização das funções que serão utilizados nos outros notebooks desse projeto.  
- Outro propósito é uma breve explicação do funcionamento individual de cada uma delas

___
### Imports iniciais
- Importa bibliotecas e funções com que trabalharemos ao longo desse notebook

In [1]:
import pandas as pd
import numpy as np

In [2]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [3]:
# Biblioteca para criação de gráficos e visualizações de dados
from matplotlib.lines import Line2D
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sn

In [4]:
# Biblioteca de Machine Learning Open source para python
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import *

___
### Dados
- Importa nossa base de dados

In [5]:
Dados=pd.read_csv('cars_Brazil_data.csv')

- Optamos por trabalhar apenas com carros que apareçam no Dataset no mínimo 500 vezes, esse valor foi escolhido de forma a separar o Dataset individual de cada carro em teste e treinamento, com 10% (no mínimo 50 linhas) do Dataset sendo direcionado para teste.   
    - <font color="#F02510"> **Observação 04/06: Talvez seja viável reduzir esse limite para 300, em contraponto o limite de 500 já fornece uma base de dados extensa o bastante e informações que parecem ser suficientes para elaborar uma conclusão.** </font>

In [6]:
#separa carros com mais de 500 ocorrências no Dataframe
Mais_de_500 = Dados['nome'].value_counts() > 500
Mais_de_500 = list(Mais_de_500.loc[Mais_de_500].index)
Dados = Dados.loc[Dados['nome'].isin(Mais_de_500),:];

#Converte as variáveis quali em quanti:
Dummies = pd.get_dummies(Dados.drop(columns='nome'), prefix=['marca', 'categoria','loc']).dropna()

#### Separando 10% do Dataframe global para treinamento
Nota para si mesmo: Estudar se faz sentido ou não eliminar as colunas nome do carro e marca 
<font color= "#DD3533">  
    
-  (Observação inicial 2 de junho: Não parece fazer)
- (Observação secundária 3 de junho: Talvez transformá-los em dummy ou hot-end)
   - Tentar Hot-End (Fiz o dummies por praticidade, checar depois. **Anotação 04/06**)
   - dummies está pronto (**Anotação 04/06**)
- Agora que `dummies` foi criado, analisar quais alterações deverão ser feitas nas funções  
(Aparentemente elas seguirão o mesmo formato visto na linha 10 da célula "Separando 10% do Dataframe de cada carro para treinamento:")
 </font>

In [7]:
# separa características dos carros, que podem ser relevantes no contexto de uma regrssão e \n
# elimina as características irrelevantes assim como a coluna com resultado esperado;
inputs = Dummies.drop(columns=["valor"])

#separa o resultado esperado, devemos calcular a previsão mais próxima a eles;
target = Dummies["valor"]

#separando dataframes teste e treino
X_train,X_test,Y_train,Y_test=tts(inputs,target,test_size=0.9)

#### Separando 10% do Dataframe de cada carro para treinamento:

In [8]:
Carro         = {}
Carro_inputs  = {}
Carro_target  = {}
Carro_X_train = {}
Carro_X_test  = {}
Carro_Y_train = {}
Carro_Y_test  = {}
Carro_Y_predict = {}

for x in Mais_de_500:
    Carro[x] = Dummies.loc[Dados['nome'] == x]
        
    Carro_inputs[x] = Carro[x].drop(columns=["valor"])
    Carro_target[x] = Carro[x]["valor"]
        
    #separando dataframes teste e treino
    Carro_X_train[x],Carro_X_test[x],Carro_Y_train[x],Carro_Y_test[x]=tts(Carro_inputs[x],Carro_target[x],test_size=0.9)

In [9]:
#Usar essa informação com a função interact do ipywidgets
def Separa_dataframe_por_carro(x):
    return (Carro_X_train[x],Carro_X_test[x],Carro_Y_train[x],Carro_Y_test[x])

___
### Modelos de Regressão e Classificação pré-processados do sklearn:
Após observação foi decido na proxima versão alterar o plot, tirando ele do return e internalizando-o na função   
<font color="#FF2515">Alterar essa célula aós estar feito para marcar melhoria</font>

#### Linear Regression (da Biblioteca SKLearn)

In [10]:
from sklearn.linear_model import LinearRegression

In [11]:
def Linear_Regression_individual(x):
    model = LinearRegression()
    model.fit(Separa_dataframe_por_carro(x)[0],Separa_dataframe_por_carro(x)[2])
    
    Carro_Y_predict = {}
    Carro_Y_predict[x] = model.predict(Carro_X_test[x])
    
    r_2 = model.score(Separa_dataframe_por_carro(x)[1],Separa_dataframe_por_carro(x)[3])
    MSE  = mean_squared_error (Separa_dataframe_por_carro(x)[3], Carro_Y_predict[x])
    MAE  = mean_absolute_error(Separa_dataframe_por_carro(x)[3], Carro_Y_predict[x])
    
    #Função de plotagem com seaborn extraída de:`https://onestopdataanalysis.com/scatter-plot-python/`
    sn.set(color_codes=True)
    plt.figure(1, figsize=(6, 4))
    plt.title("Comparação Valor real X Previsão")
    scatter = sn.scatterplot(Separa_dataframe_por_carro(x)[3],Carro_Y_predict[x], color="#D02515")
    scatter.set(xlabel="Valor Real", ylabel="Previsão")
    
    print ("R-squared: {0:.4f}".format(r_2))
    print ("Mean Squared Error: {0:.2f}" .format(MSE))
    print ("Mean Absolute Error: {0:.2f}".format(MAE))
    
    return

___
#### Regression Tree (da Biblioteca SKLearn)

In [12]:
from sklearn.tree import DecisionTreeRegressor

- Avisos:
    - Apesar do valor default do `random_state` ser `None`, ainda haviam varaiações de resultados quando o modelo era rodado(sem alterar a base de treinamento), isso foi solucionado ao trocar o valor  do `random_state` por 0.

In [13]:
def Regression_Tree_individual(x):
    model = DecisionTreeRegressor(max_depth=12,random_state=0)
    model.fit(Separa_dataframe_por_carro(x)[0],Separa_dataframe_por_carro(x)[2])
    
    Carro_Y_predict = {}
    Carro_Y_predict[x] = model.predict(Carro_X_test[x])
    
    r_2 = model.score(Separa_dataframe_por_carro(x)[1],Separa_dataframe_por_carro(x)[3])
    MSE  = mean_squared_error (Separa_dataframe_por_carro(x)[3], Carro_Y_predict[x])
    MAE  = mean_absolute_error(Separa_dataframe_por_carro(x)[3], Carro_Y_predict[x])
    
    #Função de plotagem com seaborn extraída de:`https://onestopdataanalysis.com/scatter-plot-python/`
    sn.set(color_codes=True)
    plt.figure(1, figsize=(6, 4))
    plt.title("Comparação Valor real X Previsão")
    scatter = sn.scatterplot(Separa_dataframe_por_carro(x)[3],Carro_Y_predict[x], color="#D02515")
    scatter.set(xlabel="Valor Real", ylabel="Previsão")
    
    print ("R-squared: {0:.4f}".format(r_2))
    print ("Mean Squared Error: {0:.2f}" .format(MSE))
    print ("Mean Absolute Error: {0:.2f}".format(MAE))
    
    return

___
#### Random Forest Regressor (da Biblioteca SKLearn)

In [14]:
from sklearn.ensemble import RandomForestRegressor

- Avisos:
    - Apesar do valor default do `random_state` ser `None`, ainda haviam varaiações de resultados quando o modelo era rodado(sem alterar a base de treinamento), isso foi solucionado ao trocar o valor  do `random_state` por 0.

In [15]:
def Random_Forest_individual(x):
    model = RandomForestRegressor(max_depth=12,n_estimators=50,random_state=0)
    model.fit(Separa_dataframe_por_carro(x)[0],Separa_dataframe_por_carro(x)[2])
    
    Carro_Y_predict = {}
    Carro_Y_predict[x] = model.predict(Carro_X_test[x])
    
    r_2 = model.score(Separa_dataframe_por_carro(x)[1],Separa_dataframe_por_carro(x)[3])
    MSE  = mean_squared_error (Separa_dataframe_por_carro(x)[3], Carro_Y_predict[x])
    MAE  = mean_absolute_error(Separa_dataframe_por_carro(x)[3], Carro_Y_predict[x])
    
    #Função de plotagem com seaborn extraída de:`https://onestopdataanalysis.com/scatter-plot-python/`
    sn.set(color_codes=True)
    plt.figure(1, figsize=(6, 4))
    plt.title("Comparação Valor real X Previsão")
    scatter = sn.scatterplot(Separa_dataframe_por_carro(x)[3],Carro_Y_predict[x], color="#D02515")
    scatter.set(xlabel="Valor Real", ylabel="Previsão")
    
    print ("R-squared: {0:.4f}".format(r_2))
    print ("Mean Squared Error: {0:.2f}" .format(MSE))
    print ("Mean Absolute Error: {0:.2f}".format(MAE))
    
    return

___
<font color="#FF2515">  
## Disclaimer:

Os trechos de código da seção a seguir foram extraídos de e/ou são fortemente inspirados em:
- `https://github.com/VonIgnia/C.Dados-P3/blob/master/Classificador2/_Classificadores.ipynb` 
</font>  
#### Comparação de Modelos:

In [16]:
def Modelos_de_Regressão(tipo, k=False, arvore=False, floresta=False):
    #k representa a profundidade da DecisionTree e/ou Random Forest
    if arvore:
        model = tipo(max_depth=k,random_state=0)
    elif floresta:
        model = tipo(max_depth=k,n_estimators=50,random_state=0)
    else:
        model = tipo()

    model.fit(X_train,Y_train)
    Y_predict = model.predict(X_test)
    
    r_2  = model.score(X_test,Y_test)
    MSE  = mean_squared_error (Y_test,Y_predict)
    MAE  = mean_absolute_error(Y_test,Y_predict)

    sn.set(color_codes=True)
    plt.figure(1, figsize=(9, 6))
    plt.title("Comparação Valor real X Previsão")
    scatter = sn.scatterplot(Y_test,Y_predict, color="#D02515")
    scatter.set(xlabel="Valor Real", ylabel="Previsão")

    print ("R-squared: {0:.4f}".format(r_2))
    print ("Mean Squared Error: {0:.2f}" .format(MSE))
    print ("Mean Absolute Error: {0:.2f}".format(MAE))
    
    return